<center><span style="font-size:30px;"><b>PLANT PATHOLOGY USING TRANSFER LEARNING (ResNet50)</span>

<span style="font-size:28px;">TABLE OF CONTENTS:
    
------------------------------------------------------------------------------------------------------    
    
* <span style="font-size:26px;">Import Required Libraries
* <span style="font-size:26px;">Load the Images (Train_images), DataFrame (train.csv)
* <span style="font-size:26px;">Data Augumentation using ImageDataGenerator
* <span style="font-size:26px;">Create Model (ResNet50)
* <span style="font-size:26px;">Create Callbacks
* <span style="font-size:26px;">Fit the model
* <span style="font-size:26px;">Test the Model
* <span style="font-size:26px;">Make the Submission
    

-------------

## 1. IMPORT LIBRARIES

In [ ]:
#IMPORT REQUIRED LIBRARIES:

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd

## 2. DATASET LOCATION

In [ ]:
TRAIN_PATH = "../input/plant-pathology-2021-fgvc8/train_images"
train_df = pd.read_csv("../input/plant-pathology-2021-fgvc8/train.csv")

In [ ]:
count_dict = train_df.labels.value_counts()
classes = list(count_dict.index)
classes_count = list(count_dict.values)
print("Number of unique labels: ",len(classes))
print("-------------------------------------------")
train_df.head()

-----

## 3. VISUALIZATION

In [ ]:
plt.figure(figsize=(35,15))
plt.bar(classes,classes_count)
plt.title("Number of instances per class",fontweight="bold",fontsize=40)
plt.xlabel("Classes",fontsize = 30)
plt.xticks(rotation=20,fontsize = 20,fontweight = "bold")
plt.xticks(fontsize = 20,fontweight = "bold")
plt.ylabel("Count",fontsize=30)
plt.show()

-----

## 4. DATA AUGMENTATION

In [ ]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1/255.,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split = 0.2,
    zoom_range = 0.2,
    shear_range = 0.2,
    vertical_flip = False)



In [ ]:
HEIGHT = 124
WIDTH=124
SEED = 143
BATCH_SIZE=32
train_ds = train_datagen.flow_from_dataframe(
    train_df,
    directory = TRAIN_PATH,
    x_col = "image",
    y_col = "labels",
    target_size = (HEIGHT,WIDTH),
    class_mode='categorical',
    batch_size = BATCH_SIZE,
    subset = "training",
    shuffle = True,
    seed = SEED,
    validate_filenames = False
)

In [ ]:
val_ds = train_datagen.flow_from_dataframe(
    train_df,
    directory = TRAIN_PATH,
    x_col = "image",
    y_col = "labels",
    target_size = (HEIGHT,WIDTH),
    class_mode='categorical',
    batch_size = BATCH_SIZE,
    subset = "validation",
    shuffle = True,
    seed = SEED,
    validate_filenames = False
)

-----

## 5. CREATE THE RESNET MODEL

In [ ]:
import tensorflow
def create_model():
    
    pretrained = ResNet50(include_top=False, weights='imagenet',input_shape=[HEIGHT,WIDTH, 3])
            
    x = pretrained.output
    x = tensorflow.keras.layers.GlobalAveragePooling2D() (x)
    outputs = tensorflow.keras.layers.Dense(len(classes),activation="softmax", dtype='float32')(x)
        
    model = tensorflow.keras.Model(pretrained.input, outputs)
    return model

model = create_model()

In [ ]:
model.summary()

-----

## 6. COMPILE THE MODEL

In [ ]:
import tensorflow_addons as tfa

def compile_model(model, lr=0.0001):
    
    optimizer = tensorflow.keras.optimizers.Adam(lr=lr)
    
    loss = tensorflow.keras.losses.CategoricalCrossentropy()
    
    #tf.keras.metrics.CategoricalAccuracy(name='categorical_accuracy')
    metrics = [
    tfa.metrics.F1Score(num_classes = len(classes),average = "macro",name = "f1_score")
    ]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

------

## 7. CREATE CALLBACKS

In [ ]:
METRIC = "val_f1_score"

def create_callbacks(metric = METRIC):
    
    cpk_path = './best_model.h5'
    
    checkpoint = tensorflow.keras.callbacks.ModelCheckpoint(
        filepath=cpk_path,
        monitor= metric,
        mode='max',
        save_best_only=True,
        verbose=1,
    )

    reducelr = tensorflow.keras.callbacks.ReduceLROnPlateau(
        monitor= metric,
        mode='max',
        factor=0.1,
        patience=3,
        verbose=0
    )

    earlystop = tensorflow.keras.callbacks.EarlyStopping(
        monitor= metric,
        mode='max',
        patience=10, 
        verbose=1
    )
    
    callbacks = [checkpoint, reducelr, earlystop]         
    
    return callbacks

-------

## 8. FIT THE MODEL

In [ ]:
EPOCHS= 3
VERBOSE =1

tensorflow.keras.backend.clear_session()

with tensorflow.device('/device:GPU:0'):
    
    model = create_model()
    model = compile_model(model, lr=0.0001)
   
    callbacks = create_callbacks()
    
    history = model.fit(
                        train_ds,
                        epochs=EPOCHS,
                        callbacks=callbacks,
                        validation_data = val_ds,
                        verbose=VERBOSE
                       )

------

## 9. SUBMISSION OF FILE

In [ ]:
submission = pd.read_csv('/kaggle/input/plant-pathology-2021-fgvc8/sample_submission.csv')
submission.head()

In [ ]:
test_datagen = ImageDataGenerator(
    rescale = 1./255
)
INPUT_SIZE = (124,124,3)
test_generator =  test_datagen.flow_from_dataframe(
    submission,
    directory="../input/plant-pathology-2021-fgvc8/test_images",
    x_col='image',
    y_col=None,
    class_mode=None,
    target_size=INPUT_SIZE[:2]
)

In [ ]:
train_ds.class_indices.items()

In [ ]:
def get_key(val):
    for key, value in train_ds.class_indices.items():
        if val == value:
            return key

In [ ]:
preds = model.predict(test_generator)
print(preds)

In [ ]:
def get_preds_to_labels(preds):
    pred_lists = []
    for pred in preds:
        pred_list = []
        health = (pred>=0.4)
        
        # get healthy
        if health.sum()==0:
            label = 'healthy'
            pred_list.append(label)
            
        elif pred[2]>=0.5:
            label = 'healthy'
            pred_list.append(label)
            
        # get eles label
        else:
            for j, sub in enumerate(pred):
                if sub>=0.28:
                    label = get_key(j)
                    pred_list.append(label)
                               
        pred_lists.append(' '.join(pred_list))
    return pred_lists

In [ ]:
preds_list = get_preds_to_labels(preds)

In [ ]:
submission['labels'] = preds_list
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)